In [1]:
import numpy as np
import matplotlib.pyplot as plt
from iminuit import cost, Minuit
from csaps import csaps
from scipy.signal import butter, freqs
from argparse import Namespace

%config InlineBackend.figure_formats = ['svg']

In [2]:
path = '../../rltest_data/'

# cryo model

In [20]:
class ModelValues:
    required = [
            'T_bath',  # mK
            'Rs',  # Ohm
            'L', # H 
            'sample_frequency',  # Hz
            'record_length', 
            'An',  # fitted to SEV with height 1
            'At',  # fitted to SEV with height 1
            'tau_n',  # s
            'tau_in',  # s
            'tau_t',  # s
            'An_tp',  # fitted to SEV with height 1
            'At_tp',  # fitted to SEV with height 1
            'tau_n_tp',  # s
            'tau_in_tp',  # s
            'tau_t_tp',  # s
            'bias',  # muA
            'v_to_muA',  # muA/V
            'v_set',  # V
            'Vph_calibration',  # V, if saturated then truncated fit!
            'energy',  # keV
            'T_op',  # mK
            'dRdT_op',  # Ohm/mK
            'abs_material',  # string
            'abs_volume',  # mm^3
            'tes_thickness',  # mm
            'tes_surface',  # mm^2
            'tes_width',  # mK
            'tes_rt0',  # Ohm
            'tau_holder',  # s
            'volume_holder',  # mm^3
            'heater_current',  # muA
            'pulser_amplitude',  # %
            'open_loop_out',  # 0-10
            'cpe_factor',  # keV/TPA
            'johnson_fitfreq',  # 1/s
            'thermal_fitfreq',  # 1/s
            'lowpass',  # Hertz
            'flicker_slope',  # power of the flicker component
            'squid_noise',  # pA/sqrt(Hertz)
            'nps',  # the noise power spectrum as 1D numpy array after DFT (not scaled, without prefactor, numpy normalization "backward")
                   ]
    
    returned = [
        'C',  # pJ / mK, is defined later bec mutable
        'Gb',  # pW / mK
        'G',  # heat cond between components, pW / mK
        'lamb',  # thermalization time (s)
        'eps',  # share thermalization in components
        'delta',  # share thermalization in components
        'Rs',  # Ohm
        'Rh',  # Ohm
        'L',  # H
        'Rt0',  # Ohm
        'k',  # 1/mK
        'Tc',  # mK
        'Ib',  # muA
        'dac',  # V
        'pulser_scale',  # scale factor
        'heater_current',  # muA
        'tes_flag',  # which component is a tes
        'heater_flag',  # which component has a heater
        'i_sq',  # squid noise, pA / sqrt(Hz)
        'tes_fluct',  # nWs
        'emi',  # list, nWs
        'Tb',  # function that takes one positional argument t, returns Tb in mK
        'tau',  # s, the electronics delay
                ]
    
    kB = 1.380649e-17  # mm ^ 2 * g / s^2 / mK     # nJ / mK
    na = 6.02214076e23  # number of items per mole
    h_const = 6.62607015e-34  # kg * m^2 / s
    e_charge = 1.60217663e-19  # coulombs
    density = {'W': 0.01935, 'Au': 0.01932, 'Al': 0.002702, 'CaWO4': 0.00606, 'Si': 0.00233, 'Al2O3': 0.00398, 'LiAlO2': 0.002615, 'Cu': 0.0089, }  # g/mm^3
    molar_mass = {'W': 183.84, 'Au': 196.96655, 'Al': 26.981538, 'CaWO4': 287.9156, 'Si': 28.0855, 'Al2O3': 101.961276, 'LiAlO2': 65.92, 'Cu': 63.546, }  # g/mole
    sommerfeld_constant = {'W': 1.01e3, 'Au': 0.729e3, 'Al': 1.356e3, 'Cu': 0.7e3, }  # pJ / mol / mK^2 ; gold value is from Florians book
    fermi_temp = {'W': 27_000_000, 'Au': 63_900_000, 'Al': 134_900_000, 'Cu': 8_160_000, }  # mK
    debye_frequency = {'CaWO4': 4.7, 'Si': 13.5, 'Al2O3': 21.7,  # from pantic thesis , THz
                   'LiAlO2': 8.943, 'Cu': 7.22, }  # calc above
                # mK  # debye_temp * h_const / kB
    debye_temp = {'W': 383_000, 'Au': 162_000, 'Al': 428_000, 'CaWO4': 228_000, 'Si': 648_000, 'Al2O3': 1_041_000,  # from pantic thesis 
                  'LiAlO2': 429_202, 'Cu': 346_328, }  # mK  # calc above
    
    def __init__(self, pars: dict, errors: dict):
        self.set_pars(pars, errors)
        self.pars['t'] = np.arange(0, pars['record_length'])/pars['sample_frequency']
        self.pars['freq'] = np.fft.rfftfreq(pars['record_length'], d=1/pars['sample_frequency'])
                    
        self.pars['energy'] *= 0.000160218  # to pJ
        
        self.pars['Afast'] = self.pars['An']
        self.pars['Aslow'] = self.pars['At']
        self.pars['tau_rise'] = self.pars['tau_n']
        self.pars['tau_fast'] = self.pars['tau_in']
        self.pars['tau_slow'] = self.pars['tau_t']
        
        self.pars['Afast_tp'] = self.pars['An_tp']
        self.pars['Aslow_tp'] = self.pars['At_tp']
        self.pars['tau_rise_tp'] = self.pars['tau_n_tp']
        self.pars['tau_fast_tp'] = self.pars['tau_in_tp']
        self.pars['tau_slow_tp'] = self.pars['tau_t_tp']
                
        
    def set_pars(self, pars: dict, errors: dict):
        excepted = ['abs_material']
        for name in self.required:
            assert name in pars and name in errors or name in excepted, '{} must be in pars and in errors!'.format(name)
        self.pars = pars
        self.errors = errors
        
    def load_nps(self, nps):
        self.nps = nps
        
    def sample_pars(self, ):
        new_pars = {}
        for k in self.errors.keys():
            new_pars[k] = pars[k] + errors[k]*np.random.normal()
        return new_pars
    
    def get_dump(self,):
        dump = Namespace()
        for name in self.pars.keys():
            exec("dump.{} = self.pars['{}']".format(name, name))
        return dump
    
    def save_dump(self, dump):
        for name in dump.__dict__.keys():
            self.pars[name] = eval('dump.{}'.format(name))
        
    def calc_values(self, store=True, use_part=True, **kwargs):
        if not 'pars' in kwargs:
            pars = self.pars
        else:
            pars = kwargs['pars']
            
        values = {}
        
        dump = self.get_dump()
            
        # calc V_op, R_op
        dump.V_op = dump.bias / dump.v_to_muA / (dump.Rs / dump.tes_rt0 + 1) - dump.v_set
        dump.R_op = dump.Rs / (dump.bias / dump.v_to_muA / dump.V_op - 1)
        
        # calc R_calibration, Tph_calibration
        dump.R_calibration = dump.Rs / (dump.bias / dump.v_to_muA / dump.V_op - 1) + dump.v_to_muA * dump.Rs * dump.bias / (dump.bias - dump.v_to_muA * dump.V_op) ** 2 * dump.Vph_calibration 
        dump.Tph_calibration = (dump.R_calibration - dump.R_op) / dump.dRdT_op
        
        # calc copper holders
        mat = 'Cu'
        n_mol = dump.volume_holder * self.density[mat] / self.molar_mass[mat]
        dump.C_holder = 12 * np.pi ** 4 / 5 * self.na * self.kB * (dump.T_op / self.debye_temp[mat]) ** 3 * n_mol * 1e3
        dump.G_holder = dump.C_holder/dump.tau_holder
        
        # calc Cph, Ce
        mat = dump.abs_material
        n_mol = dump.abs_volume * self.density[mat] / self.molar_mass[mat] 
        dump.Cph = 12 * np.pi ** 4 / 5 * self.na * self.kB * (dump.T_op / self.debye_temp[mat]) ** 3 * n_mol * 1e3

        n_mol = dump.tes_surface * dump.tes_thickness * self.density['W'] / self.molar_mass['W']
        dump.Ce = self.sommerfeld_constant['W'] * dump.T_op * (2.43 - 1.43 * dump.R_op / dump.tes_rt0) * n_mol 
        
        # calc tau_film, tau_crystal, epsilon  - assuming calorimetric mode !!
        dump.epsilon = + dump.Afast * dump.Tph_calibration * dump.Ce / dump.energy
        dump.tau_crystal = + dump.tau_rise / dump.epsilon
        dump.tau_film = dump.tau_rise / (1 - dump.epsilon)
        
        # calc G_eff, absorber_bath, eph_coupling
        dump.G_eff = dump.Ce / dump.tau_fast
        dump.absorber_bath = dump.Cph / dump.tau_slow
        num = dump.G_eff*dump.absorber_bath*(dump.tau_rise + dump.tau_slow) - dump.Cph*dump.G_eff - dump.Ce*dump.absorber_bath
        denom = dump.Cph + dump.Ce - (dump.tau_rise + dump.tau_slow)*(dump.G_eff + dump.absorber_bath)
        dump.eph_coupling = num / denom

        # calc G_etf, I_f, G_tot, thermal_link, G_comb
        dump.I_f = dump.bias * dump.Rs / (dump.Rs + dump.R_op) 
        dump.G_etf = dump.I_f**2 * dump.dRdT_op * (dump.R_op - dump.Rs)/(dump.R_op + dump.Rs) 
        dump.G_tot = dump.G_eff + (dump.eph_coupling * dump.absorber_bath) / (dump.eph_coupling + dump.absorber_bath)

        dump.thermal_link = dump.G_eff - dump.G_etf
        dump.G_comb = dump.G_tot - dump.G_etf
    
        # calc delta, heater_resistance, absorber_temp
        if dump.tau_n_tp < dump.tau_in_tp:  # calorimetric
            dump.delta = + dump.Afast_tp * dump.Tph_calibration * dump.Ce / dump.energy
        else:  # bolometric
            dump.delta = - dump.Afast_tp * dump.Tph_calibration * (dump.eph_coupling + dump.G_eff) * dump.tau_fast_tp / dump.energy 
        tau_crystal_tp = + dump.tau_rise_tp / dump.delta
        tau_film_tp = dump.tau_rise_tp / (1 - dump.delta)

        dump.I_h = dump.heater_current * np.sqrt(dump.open_loop_out/ 10) 

        G_eb = dump.thermal_link  # names are shorter for calculation below
        G_ea = dump.eph_coupling
        G_ab = dump.absorber_bath

        R_H_times_I_H_squared = ((G_ea + G_ab) * (G_ea * dump.T_op + G_eb * (dump.T_op - dump.T_bath) - dump.R_op * dump.I_f**2) - G_ea * (G_ea * dump.T_op + G_ab * dump.T_bath)) 
        R_H_times_I_H_squared /= (dump.delta * (G_ea + G_ab) + (1 - dump.delta) * G_ea)

        dump.heater_resistance = R_H_times_I_H_squared / dump.I_h ** 2

        dump.absorber_temp = ((1 - dump.delta) * (G_ea * dump.T_op + G_eb * (dump.T_op - dump.T_bath) - dump.R_op * dump.I_f**2) +  dump.delta * (G_ea * dump.T_op + G_ab * dump.T_bath))
        dump.absorber_temp /= (dump.delta * (G_ea + G_ab) + (1 - dump.delta) * G_ea)

        I_h = dump.heater_current * np.sqrt(dump.pulser_amplitude)
        if dump.tau_n_tp < dump.tau_in_tp:  # calorimetric
            dump.pulser_correction = dump.Afast_tp * dump.Ce * dump.cpe_factor * dump.Tph_calibration
            dump.pulser_correction /= (dump.heater_resistance * dump.I_h ** 2 * dump.energy * dump.delta * dump.pulser_amplitude)
        else:  # bolometric
            dump.pulser_correction = - dump.Afast_tp * dump.tau_fast_tp * (dump.thermal_link + dump.eph_coupling) * dump.Ce * dump.cpe_factor * dump.Tph_calibration
            dump.pulser_correction /= (dump.heater_resistance * dump.I_h ** 2 * dump.energy * dump.delta * dump.pulser_amplitude)
        
        # calc ex_jf, ex_phonon, tes_fluct 
        dump.norm_factor = 2 / dump.sample_frequency / dump.record_length / 0.875
        dump.nps *= dump.norm_factor * dump.v_to_muA ** 2
        
        self.save_dump(dump)
        
        noise, info = self.noise_template(use_part)
        
        dump = self.get_dump()

        # build return vars
        C = np.array([dump.Ce, dump.Cph, dump.C_holder])
        Gb = np.array([dump.thermal_link, dump.absorber_bath, 0.])
        G = np.array([[0., dump.eph_coupling, 0.], 
                      [dump.eph_coupling, 0, dump.G_holder], 
                      [0., dump.G_holder, 0.], ])
        lamb = dump.tau_rise
        eps = np.array([[dump.epsilon, 1 - dump.epsilon, 0.], 
                        [dump.epsilon, 1 - dump.epsilon, 0.],
                        [dump.epsilon, 1 - dump.epsilon, 0.]])
        delta = np.array([[dump.delta, 1 - dump.delta, 0.], 
                        [dump.delta, 1 - dump.delta, 0.],
                        [dump.delta, 1 - dump.delta, 0.]])
        Rs = np.array([dump.Rs])
        Rh = np.array([dump.heater_resistance])
        L = np.array([dump.L])
        Rt0 = np.array([dump.tes_rt0])
        k = np.array([dump.tes_width/2])
        Tc = np.array([dump.T_op])
        Ib = np.array([dump.bias])
        dac = np.array([dump.open_loop_out])
        pulser_scale = np.array([dump.pulser_amplitude*dump.pulser_correction])
        heater_current = np.array([dump.heater_current])  # TODO
        tes_flag = np.array([True, False, False], dtype=bool)
        heater_flag = np.array([False, True, False], dtype=bool)
        i_sq = np.array([dump.squid_noise])
        tes_fluct = np.array([dump.tes_fluct])
        emi = np.array([[dump.p0, dump.p1, dump.p2],])
        Tb = lambda t: dump.T_bath
        tau = np.array([1])
        
        for name in self.returned:
            values[name] = eval(name)
            
        self.save_dump(dump)
        
        if store:
            self.values = values
        return values
    
    def noise_transfer_functions_part(self, ):  # like Irwin
        
        dump = self.get_dump()
        
        dump.norm_factor = 2 / dump.sample_frequency / dump.record_length / 0.875
        
        I_f0 = dump.I_f
        R_f0 = dump.R_op
        R_s = dump.Rs
        tau = dump.Ce / dump.thermal_link
        tau_el = dump.L / (R_f0 + R_s)
        L_I = I_f0 ** 2 / dump.thermal_link * dump.dRdT_op  # loop gain
        w = dump.freq
        tau_I = tau / (1 - L_I)

        # ... for thermal noise (1/muA/Ohm)
        dump.s21_int = -1 / (I_f0 * R_f0) 
        denom = dump.L / (tau_el * R_f0 * L_I)
        denom += (1 + R_s / R_f0)  # here the sign other than in Irwin (theirs voltage biased)
        denom += 2 * np.pi * 1j * w * dump.L * tau / (R_f0 * L_I) * (1 / tau + 1 / tau_el) 
        denom -= (4 * np.pi**2 * w**2 * tau * dump.L) / L_I / R_f0
        dump.s21_int /= denom 

        # ... for TES johnson and flicker (1/Ohm)
        dump.s21_ext = -1 / (I_f0 * R_f0) 
        denom = dump.L * tau / (tau_el * tau_I * R_f0 * L_I) + 2 
        denom += 2 * np.pi * 1j * w * dump.L * tau / (R_f0 * L_I) * (1 / tau_I + 1 / tau_el)
        denom -=  (4 * np.pi**2 * w**2 * tau) / L_I * dump.L / R_f0
        dump.s21_ext /= denom
        dump.s22_ext = dump.s21_ext * I_f0 * (L_I - 1) / L_I * (1 + 2 * np.pi * 1j * w * tau_I)

        # ... for shunt johnson noise and emi (1/Ohm)
        dump.s22_int = -dump.s21_int * I_f0 * (1 / L_I) * (1 + 2 * np.pi * 1j * w * tau)   
        
        
        self.save_dump(dump)

        return dump.s21_int, dump.s22_ext, dump.s22_int
        
    def noise_transfer_functions_tot(self, ):  # like Pantic
        pass
        
    def noise_template(self, use_part=True): 
        
        dump = self.get_dump()
    
        dump.norm_factor = 2 / dump.sample_frequency / dump.record_length / 0.875    
    
        info = {}
        
        # build lowpass
        
        b, a = butter(N=1, Wn=dump.lowpass, btype='lowpass', analog=True)
        _, h = freqs(b, a, worN=dump.freq)

        dump.h = np.abs(h) ** 2
        
        # build prefactors
        
        freq = dump.freq
        dump.thermal_pref = 4 * self.kB * dump.T_op**2 * dump.G_eff * 2/5 * (1 - (dump.T_bath/dump.T_op)**5) / (1 - (dump.T_bath/dump.T_op)**2) * 1e3 * dump.h  # pJ
        dump.jf_pref = 4 * self.kB * dump.T_op * dump.R_op * 1e3 * dump.h  # pJ * Ohm
        dump.js_pref = 4 * self.kB * dump.T_bath * dump.Rs * 1e3 * dump.h  # pJ * Ohm
        
        # build lowpass and emi spikes
        
        power_ts = np.sin(2 * np.pi * self.pars['t'] * 50)
        power_component = np.abs(np.fft.rfft(power_ts)) ** 2 * dump.norm_factor * dump.v_to_muA ** 2

        power_ts_two = np.sin(2 * np.pi * self.pars['t'] * 150)
        power_component_two = np.abs(np.fft.rfft(power_ts_two)) ** 2 * dump.norm_factor * dump.v_to_muA ** 2

        power_ts_three = np.sin(2 * np.pi * self.pars['t'] * 250)
        power_component_three = np.abs(np.fft.rfft(power_ts_three)) ** 2 * dump.norm_factor * dump.v_to_muA ** 2

        # build the transfer functions
        
        if use_part:
            dump.s21_int, dump.s22_ext, dump.s22_int = self.noise_transfer_functions_part()
        else:
            dump.s21_int, dump.s22_ext, dump.s22_int = self.noise_transfer_functions_tot()

        # calc the expected templates
        
        js_noise = dump.js_pref * np.abs(dump.s22_int)**2
        s_noise = (dump.squid_noise) ** 2 * np.ones(freq.shape) * 1e-12 * dump.h
        j_noise = dump.jf_pref * np.abs(dump.s22_ext)**2         
        t_noise = dump.thermal_pref * np.abs(dump.s21_int)**2 
        flicker = 1 / np.maximum(freq,1) ** (dump.flicker_slope) * dump.I_f ** 2 * dump.R_op ** 2 * np.abs(dump.s22_ext)**2
        flicker = flicker * dump.h
        idx0 = (np.abs(self.pars['freq'] - 50)).argmin()
        idx1 = (np.abs(self.pars['freq'] - 100)).argmin()
        idx2 = (np.abs(self.pars['freq'] - 150)).argmin()
        dump.p0 = np.sqrt(dump.nps[idx0,1]/power_component[idx0])
        dump.p0_norm = dump.p0 ** 2 * power_component /np.abs(dump.s22_int[idx0])**2
        dump.p1 = np.sqrt(dump.nps[idx1,1]/power_component_two[idx1])
        dump.p1_norm = dump.p1 ** 2 * power_component_two /np.abs(dump.s22_int[idx1])**2
        dump.p2 = np.sqrt(dump.nps[idx2,1]/power_component_three[idx2])
        dump.p2_norm = dump.p2 ** 2 * power_component_three /np.abs(dump.s22_int[idx2])**2
        em_noise = (dump.p0_norm + dump.p1_norm + dump.p2_norm) * np.abs(dump.s22_int)**2
        
        # calc and correct for excess values
        
        jfq = int(dump.johnson_fitfreq)
        tfq = int(dump.thermal_fitfreq)
        dump.ex_js = np.sqrt(dump.nps[jfq,1]/(js_noise[jfq] + s_noise[jfq]))
        dump.ex_phonon = np.sqrt(dump.nps[tfq,1]/(t_noise[tfq] + j_noise[tfq] + s_noise[tfq]))
        dump.tes_fluct = np.sqrt(dump.nps[1,1]/(flicker[1] + s_noise[1]))
    
        js_noise *= dump.ex_js ** 2 
        t_noise *= dump.ex_phonon ** 2
        flicker *= dump.tes_fluct ** 2
    
        # wrap up and return
    
        for name in ['js_pref', 'jf_pref', 'thermal_pref', 'ex_js', 'ex_phonon', 'tes_fluct']:
            info[name] = eval('dump.' + name) 
        
        noise = js_noise + s_noise + t_noise + j_noise + flicker + em_noise
        
        self.save_dump(dump)
        
        return noise, info
    
    def single_pulse(t, t0, A, tau_rise, tau_decay):
        n = t.shape[0]
        cond = t > t0
        pulse = np.zeros(n)
        pulse[cond] = A * (np.exp(-(t[cond] - t0) / tau_decay) - np.exp(-(t[cond] - t0) / tau_rise))
        return pulse    
    
    def plot_pulse(self,):  # useful for debugging
        
        An_calc = ...
        At_calc = ...
        tau_n_calc = ...
        tau_in_calc = ...
        tau_t_calc = ...
        
        # assuming calorimetric!
        fast_pulse_meas = single_pulse(dump.t, 
                              0, 
                              dump.An,
                              dump.tau_n,
                              dump.tau_in)
        slow_pulse_meas = single_pulse(dump.t, 
                              0, 
                              dump.At,
                              dump.tau_n,
                              dump.tau_t)
        
        plt.plot(dump.t, fast_pulse_meas + slow_pulse_meas, 
                 label='measured', color='red', linestyle='dashed')
        
        An_calc = ... # TODO
        At_calc = ...
        tau_n_calc = ...
        tau_in_calc = ...
        tau_t_calc = ...
        
        fast_pulse = single_pulse(sev[name][:,0]*1000, 
                              0., 
                              pspars_fit[name][2],
                              pspars_fit[name][4],
                              pspars_fit[name][5])
        slow_pulse = single_pulse(sev[name][:,0]*1000, 
                              0.,
                              pspars_fit[name][1] - pspars_fit[name][2],
                              pspars_fit[name][4],
                              pspars_fit[name][3])
        
        plt.plot(dump.t, slow_pulse, label='slow')
        plt.plot(dump.t, fast_pulse, label='fast')
        plt.plot(dump.t, fast_pulse + slow_pulse, 
                 color='black', label='joint')

        plt.xlim(-0.003,0.03)
        plt.legend()
        plt.xlabel('Time (s)')
        plt.ylabel('Amplitude (V)')
        plt.show()
    
    def get_pulse(self, ): # TODO
        pass
        
    def get_pulse_num(self, ): # TODO
        pass
    
    def get_nps(self, ): # TODO
        pass
        
    def noise_function(self, ):
        pass
        
    def get_noise_trace(self, ): # TODO
        pass
        
    def get_noise_trace_ou(self, ): # TODO
        pass
        
    def get_values(self, sample=False):
        if sample:
            pars = self.sample_pars()
            values = self.calc_values()
        else:
            pars = self.pars
            values = self.calc_values()
        return values

In [21]:
def get_pars_and_errors():
    
    # parameters
    
    T_bath = 15.0  # mK
    Rs = 0.040  # Ohm
    L = 3.5e-7  # H 
    sample_frequency = 25000  # Hz
    record_length = 16384 
    An = 1.04116448  # fitted to SEV with height 1
    At = 0.14316717000000012  # fitted to SEV with height 1
    tau_n = 0.00038244691  # s
    tau_in = 0.00935751479  # s
    tau_t = 0.07205835566  # s
    An_tp = 1.9114554669397188  # fitted to SEV with height 1
    At_tp = 0.1500907083857006  # fitted to SEV with height 1
    tau_n_tp = 0.004866101142065952  # s
    tau_in_tp = 0.01798445302006269  # s
    tau_t_tp = 0.18906776743488907  # s
    bias = 0.28 * 17.86  # muA
    v_to_muA = 0.17341040462427743  # muA/V
    v_set = 1.55  # V
    Vph_calibration = 0.37  # V, if saturated then truncated fit!
    energy = (5.9 * 8/9 + 1/9 * 6.4)  # keV
    T_op = 30.791  # mK
    dRdT_op = 0.1  # Ohm/mK
    abs_material = "LiAlO2"  # string
    abs_volume = 4000.0  # mm^3
    tes_thickness = 0.0002  # mm
    tes_surface = 2.04  # mm^2
    tes_width = 1.0  # mK
    tes_tc = 30.8  # mK
    tes_rt0 = 0.110  # Ohm
    tau_holder = 200.0  # s
    volume_holder = 400.0  # mm^3
    heater_current = 0.336 * 14.280  # muA
    pulser_amplitude = 0.091  # %
    open_loop_out = 1.58  # 0-10
    cpe_factor = 3.103  # keV/TPA
    johnson_fitfreq = 762.0  # 1/s
    thermal_fitfreq = 35.0  # 1/s
    lowpass = 3500
    flicker_slope = 2.5
    squid_noise = 1.9
    nps = np.loadtxt(path + 'Channel_0_NPS.xy', skiprows=3)

    pars = {var_name.strip(): var_value for var_name, var_value in locals().items() if not var_name.startswith('__') and not callable(var_value)}

    # errors
        
    T_bath = 1.50  # mK
    Rs = 0.004  # Ohm
    L = .35e-7  # H 
    sample_frequency = 0  # Hz
    record_length = 0 
    An = 0.1*1.04116448  # fitted to SEV with height 1
    At = 0.1*0.14316717000000012  # fitted to SEV with height 1
    tau_n = 0.1*0.00038244691  # s
    tau_in = 0.1*0.00935751479  # s
    tau_t = 0.1*0.07205835566  # s
    An_tp = 0.1*1.9114554669397188  # fitted to SEV with height 1
    At_tp = 0.1*0.1500907083857006  # fitted to SEV with height 1
    tau_n_tp = 0.1*0.004866101142065952  # s
    tau_in_tp = 0.1*0.01798445302006269  # s
    tau_t_tp = 0.1*0.18906776743488907  # s
    bias = 0.01*0.28 * 17.86  # muA
    v_to_muA = 0.01*0.17341040462427743  # muA/V
    v_set = 0.01*1.55  # V
    Vph_calibration = 0.01*0.37  # V, if saturated then truncated fit!
    energy = 0.01*(5.9 * 8/9 + 1/9 * 6.4)  # keV
    T_op = 1  # mK
    dRdT_op = 0.2*0.1  # Ohm/mK
    abs_volume = 0.01*4000.0  # mm^3
    tes_thickness = 0.1*0.0002  # mm
    tes_surface = 0.01*2.04  # mm^2
    tes_width = 0.1*1.0  # mK
    tes_tc = .2  # mK
    tes_rt0 = .1*0.110  # Ohm
    tau_holder = .2*200.0  # s
    volume_holder = .2*400.0  # mm^3
    heater_current = .01*0.336 * 14.280  # muA
    pulser_amplitude = .01*0.091  # %
    open_loop_out = .01*1.58  # 0-10
    cpe_factor = .01*3.103  # keV/TPA
    johnson_fitfreq = 0  # 1/s
    thermal_fitfreq = 0  # 1/s
    lowpass = 0
    flicker_slope = 0.1*2.5
    squid_noise = 0.3*1.9
    nps = 0.

    excepted = ['pars', 'abs_material']
    errors = {var_name.strip(): var_value for var_name, var_value in locals().items() if not var_name.startswith('__') and not callable(var_value) and not (var_name in excepted or var_name == 'excepted')}
    
    return pars, errors

In [22]:
pars, errors = get_pars_and_errors()

In [23]:
mv = ModelValues(pars, errors)

In [24]:
mv.get_values(True)

{'C': array([0.00177236, 0.11387951, 0.0765271 ]),
 'Gb': array([0.09718931, 1.58037891, 0.        ]),
 'G': array([[0.        , 0.21401944, 0.        ],
        [0.21401944, 0.00038264, 0.        ],
        [0.        , 0.00038264, 0.        ]]),
 'lamb': 0.00038244691,
 'eps': array([[0.09667376, 0.90332624, 0.        ],
        [0.09667376, 0.90332624, 0.        ],
        [0.09667376, 0.90332624, 0.        ]]),
 'delta': array([[0.17748165, 0.82251835, 0.        ],
        [0.17748165, 0.82251835, 0.        ],
        [0.17748165, 0.82251835, 0.        ]]),
 'Rs': array([0.04]),
 'Rh': array([4.28306214]),
 'L': array([3.5e-07]),
 'Rt0': array([0.11]),
 'k': array([0.5]),
 'Tc': array([30.791]),
 'Ib': array([5.0008]),
 'dac': array([1.58]),
 'pulser_scale': array([0.19917524]),
 'heater_current': array([4.79808]),
 'tes_flag': array([ True, False, False]),
 'heater_flag': array([False,  True, False]),
 'i_sq': array([1.9]),
 'tes_fluct': array([0.00043352]),
 'emi': array([[0.0001